In [1]:
Connect-AzAccount

To override which subscription Connect-AzAccount selects by default, use `Update-AzConfig -DefaultSubscriptionForLogin 00000000-0000-0000-0000-000000000000`. Go to https://go.microsoft.com/fwlink/?linkid=2200610 for more information.

Account                SubscriptionName TenantId                             Environment
-------                ---------------- --------                             -----------
piotr@demo.wachulec.me PAYG-01          f82b557d-79f8-4f75-8cfe-96847cd44a6a AzureCloud



In [2]:
$moduleName = "AzTable"

# Check if the module is already installed
$moduleInstalled = Get-Module -ListAvailable | Where-Object { $_.Name -eq $moduleName }

if ($moduleInstalled) {
    Write-Output "Module $moduleName is already installed."
} else {
    Write-Output "Module $moduleName is not installed. Installing..."
    Install-Module -Name $moduleName -Force
}


Module AzTable is already installed.


In [4]:
# Set the Azure storage account, resource group and table name
$storageAccountName = "jupytersa"
$resourceGroupName = "jupyter-demo"
$tableName = "LandingZonesInfo"

# Get the table reference
$storageAccount = Get-AzStorageAccount -ResourceGroupName $resourceGroupName -Name $storageAccountName
$storageContext = $storageAccount.Context
$table = (Get-AzStorageTable -Name $tableName -Context $storageContext).CloudTable

# Retrieve all rows from the table
$landingZonesInfo = Get-AzTableRow -Table $table

In [5]:
# Display the retrieved rows
$landingZonesInfo


LandingZoneOwner       : jane.doe@abcxyz.io
ReaderRoleGroupId      : bb8d73de-5bca-4c5f-9562-12b3ef7c6ec0
ContributorRoleGroupId : 8a3e28b3-ea86-4b7e-b5cb-0f93c0eb20af
CreationDate           : 2024-01-02T05:25:40.000Z
PartitionKey           : apac
RowKey                 : 0a9c56ec-0a20-4ce4-9e8d-f6faa53a6bdb
TableTimestamp         : 1/29/2024 6:27:37 AM +01:00
Etag                   : W/"datetime'2024-01-29T05%3A27%3A37.3865476Z'"

ContributorRoleGroupId : 2e11a290-5949-4298-b432-c754d34cb78c
CreationDate           : 2024-01-07T05:25:40.000Z
LandingZoneOwner       : john.doe@abcxyz.io
ReaderRoleGroupId      : 7ad4eff6-e832-41d0-8687-85a76d1d6ae1
PartitionKey           : emea
RowKey                 : 88a99f8e-abc3-4f87-b5d1-6582ecf72501
TableTimestamp         : 1/29/2024 6:27:45 AM +01:00
Etag                   : W/"datetime'2024-01-29T05%3A27%3A45.8385044Z'"



In [6]:
$subscriptions = Get-AzSubscription
$subscriptions


Name    Id                                   TenantId                             State
----    --                                   --------                             -----
PAYG-01 88a99f8e-abc3-4f87-b5d1-6582ecf72501 f82b557d-79f8-4f75-8cfe-96847cd44a6a Enabled
PAYG-02 0a9c56ec-0a20-4ce4-9e8d-f6faa53a6bdb f82b557d-79f8-4f75-8cfe-96847cd44a6a Enabled



In [8]:
$landingZonesFullInfo = $null

# Create the connected array
$landingZonesFullInfo = $landingZonesInfo | ForEach-Object {
    $landingZoneInfo = $_
    $subscription = $subscriptions | Where-Object { $_.Id -eq $landingZoneInfo.RowKey }
    $landingZoneInfo | Select-Object *, @{ Name = 'Name'; Expression = { $subscription.Name } }
}

# Display the connected array
$landingZonesFullInfo


LandingZoneOwner       : jane.doe@abcxyz.io
ReaderRoleGroupId      : bb8d73de-5bca-4c5f-9562-12b3ef7c6ec0
ContributorRoleGroupId : 8a3e28b3-ea86-4b7e-b5cb-0f93c0eb20af
CreationDate           : 2024-01-02T05:25:40.000Z
PartitionKey           : apac
RowKey                 : 0a9c56ec-0a20-4ce4-9e8d-f6faa53a6bdb
TableTimestamp         : 1/29/2024 6:27:37 AM +01:00
Etag                   : W/"datetime'2024-01-29T05%3A27%3A37.3865476Z'"
Name                   : PAYG-02

ContributorRoleGroupId : 2e11a290-5949-4298-b432-c754d34cb78c
CreationDate           : 2024-01-07T05:25:40.000Z
LandingZoneOwner       : john.doe@abcxyz.io
ReaderRoleGroupId      : 7ad4eff6-e832-41d0-8687-85a76d1d6ae1
PartitionKey           : emea
RowKey                 : 88a99f8e-abc3-4f87-b5d1-6582ecf72501
TableTimestamp         : 1/29/2024 6:27:45 AM +01:00
Etag                   : W/"datetime'2024-01-29T05%3A27%3A45.8385044Z'"
Name                   : PAYG-01

